# Activation Quantization

In [100]:
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from dongnet import dongnet12



In [101]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not abailable. Training on CPU...')
else:
    print('CUIDA is available! Training on GPU...')
    

CUDA is not abailable. Training on CPU...


In [102]:
def quantize(X, NBIT):
    # 1. find threshold
    alpha = np.max(X)
    beta = np.min(X)
    alpha_q = -2**(NBIT - 1)
    beta_q = 2**(NBIT - 1) - 1

    s = (beta - alpha) / (beta_q - alpha_q)
    z = int((beta*alpha_q - alpha * beta_q) / (beta - alpha))

    data_q = np.round(1/s * X + z, decimals=0)
    data_q = np.clip(data_q, alpha_q, beta_q)    
    data_q = data_q.astype(np.int8)
        
    data_qn = data_q
    data_qn = data_qn.astype(np.int32)
    data_qn = s * (data_qn - z)
    data_qn = data_qn.astype(np.float32)
    
    return data_qn

def Actquantize(X, NBIT=8):
    # 1. find threshold(maximu range)
    threshold = np.max([np.abs(np.max(X)), np.abs(np.min(X))])
    
    QRANGE = 2**(NBIT-1)
    # 2. find p : decimal position of quantized value
    p = np.int(np.log2((QRANGE-1)/threshold))
    
    # 3.quantize using 8bit
    # 3.1 apply threholds
    data_th = np.clip(X, -QRANGE*2**(-p), (QRANGE-1)*2**(-p))
    
    #3.2 calculate the scale factor for quantization
    SCALE = 2**(-p)
    
    # 3.3 Quantize (apply scale factor)
    data_qn = np.round(data_th/SCALE)
    
    data_dqn = data_qn * SCALE
    return data_dqn, p

def evaluate_model(model, test_loader, device):
    #batch norm 고정, dropout 안함, gradient 계산안함
    model.eval()
    #model 파라미터를 지정한 device 메모리에 올림
    model.to(device)

    running_corrects = 0
    criterion = nn.CrossEntropyLoss()

    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        #텐서의 최대값과 index를 반환
        _, preds = torch.max(outputs, 1)

        if criterion is not None:
            loss = criterion(outputs, labels).item()
        else:
            loss = 0

        # inputs.size(0) 현재 mini-batch의 크기(input의 0번째 dimension 크기)
        running_corrects += torch.sum(preds == labels.data)
    
    eval_accuracy = running_corrects / len(test_loader.dataset)

    print("Dongnet12 cifar100 Accuracy: {:.4f}".format(eval_accuracy))
    
# Accuracy Quantized model
def evaluate_qmodel(model, test_loader, device, NBIT):
    
    disallowed_layer_names = []
    # linear operation, convolution, batchnorm
    whitelist=[torch.nn.Linear, torch.nn.Conv1d, torch.nn.Conv2d, torch.nn.Conv3d, nn.BatchNorm2d]
    whitelist_layer_types = tuple(whitelist)
    eligible_modules_list = []
    eligible_param_list = []
    for name, mod in model.named_modules():
        if isinstance(mod, whitelist_layer_types) and name not in disallowed_layer_names:
            eligible_modules_list.append((name, mod))
            eligible_param_list.append(name)
    
    #1. extract weigth
    #2. quantized weight
    #3. Load quantized weight into the model
    for name, param in model.named_parameters():
        layername = '.'.join(name.split('.')[:1])
        layername2 = '.'.join(name.split('.')[:2])
        if layername in eligible_param_list:
            weight = param.cpu().detach().numpy()
            dqn = quantize(weight, NBIT)
            param.data = torch.from_numpy(dqn)
        elif layername2 in eligible_param_list:
            weight = param.cpu().detach().numpy()
            dqn = quantize(weight, NBIT)
            param.data = torch.from_numpy(dqn)
            

    #batch norm 고정, dropout 안함, gradient 계산안함
    model.eval()
    #model 파라미터를 지정한 device 메모리에 올림
    model.to(device)

    running_corrects = 0
    criterion = nn.CrossEntropyLoss()
    
    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        #텐서의 최대값과 index를 반환
        _, preds = torch.max(outputs, 1)

        if criterion is not None:
            loss = criterion(outputs, labels).item()
        else:
            loss = 0

        # inputs.size(0) 현재 mini-batch의 크기(input의 0번째 dimension 크기)
        running_corrects += torch.sum(preds == labels.data)
    eval_accuracy = running_corrects / len(test_loader.dataset)

    print("{}bit quantize model cifar100 Accuracy : {:.4f}".format(NBIT, eval_accuracy))
    


In [103]:
class NewModel(nn.Module):
    def __init__(self, weight_path, output_layers, *args):
        super().__init__(*args)
        self.output_layers = output_layers
        self.selected_out = OrderedDict()
        self.pretrained = dongnet12()
        self.pretrained.load_state_dict(torch.load(weight_path))
        self.fhooks = []
        
        for i,l in enumerate(list(self.pretrained._modules.keys())):
            if i in self.output_layers:
                self.fhooks.append(getattr(self.pretrained, l).register_forward_hook(self.forward_hook(l)))
        
    def forward_hook(self, layer_name):
        def hook(module, input, output):
            self.selected_out[layer_name] = output
        return hook
    
    def forward(self, x):
        out = self.pretrained(x)
        return out, self.selected_out

In [104]:
weight_path = 'int8qmodel.pth'
model = NewModel(weight_path, output_layers = [i for i in range(60)]).to('cpu')

In [105]:
train_transform = transforms.Compose([
        #padding을 4추가하고 이미지를 32 X 32로 자름
        transforms.RandomCrop(32, padding=4),
        #이미지를 뒤집는다
        transforms.RandomHorizontalFlip(),
        #텐서로 바꾼다
        transforms.ToTensor(),
        #각 채널별 평균, 표준편차``
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    # CIFAR10 train 데이터 가져오기
    # We will use test set for validation and test in this project.
    # Do not use test set for validation in practice!
    # CIFAR10 test 데이터 가져오기
train_set = torchvision.datasets.CIFAR100(root="data", train=True, download=True, transform=train_transform) 
test_set = torchvision.datasets.CIFAR100(root="data", train=False, download=True, transform=test_transform)

train_sampler = torch.utils.data.RandomSampler(train_set)
test_sampler = torch.utils.data.SequentialSampler(test_set)

    #sampler은 dataset에서 데이터를 뽑아오는 역할, num_workers는 데이터를 읽어오는 프로세스 수
train_loader = torch.utils.data.DataLoader(
    dataset=train_set, batch_size=128,
    sampler=train_sampler, num_workers=8)

test_loader = torch.utils.data.DataLoader(
    dataset=test_set, batch_size=128,
    sampler=test_sampler, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


# collect p for each layer for 100 samples after weight quantization


In [110]:
count = 0
for k in layerout.keys():
    if k.startswith('leaky'):
        count += 1

quant_p = np.zeros((100, count+1))        
batch = 0
for inputs, labels in test_loader:
    if train_on_gpu:
        inputs.cuda(), labels.cuda()
    out, layerout = model(inputs)
    col = 0
    for k in layerout.keys():
        if k.startswith('leaky'):
            if batch ==0:
                print(k)
            _, p = Actquantize(layerout[k].cpu().detach().numpy())
            quant_p[batch, col] = p
            col += 1
    batch += 1
    if batch ==100:
        break

leaky_relu1
[[3. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

/tmp/ipykernel_44097/2408902859.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  p = np.int(np.log2((QRANGE-1)/threshold))


[[3. 3. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0.

[[3. 3. 3. 4. 4. 4. 3. 3. 3. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0.

[[3. 3. 3. 4. 4. 4. 3. 3. 3. 0.]
 [3. 3. 3. 4. 4. 4. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0.

...

[[3. 3. 3. 4. 4. 4. 3. 3. 3. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 2. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 3. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 3. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 3. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4.

[[3. 3. 3. 4. 4. 4. 3. 3. 3. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 2. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 3. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 3. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 3. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4.

[[3. 3. 3. 4. 4. 4. 3. 3. 3. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 2. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 3. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 3. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 3. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4.

[[3. 3. 3. 4. 4. 4. 3. 3. 3. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 2. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 3. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 3. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 3. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4.

[[3. 3. 3. 4. 4. 4. 3. 3. 3. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 2. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 3. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 3. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 3. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 3. 4. 4. 0.]
 [3. 3. 4. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 3. 4. 4. 4. 4. 3. 4. 0.]
 [3. 3. 4.

In [107]:
# find most frequent
def mode(X):
    return max(set(X), key=X.count)

cfg = []
for i in range(quant_p.shape[1]):
    print(i, mode(list(quant_p[:,i])))
    cfg.append(int(mode(list(quant_p[:,i]))))

0 3.0
1 3.0
2 4.0
3 4.0
4 4.0
5 4.0
6 4.0
7 4.0
8 4.0
9 0.0


# build model with static quantization, whic applied after activation

In [108]:
class dongnet12q(nn.Module):
    
    def __init__(self, p_list):
        super(dongnet12q, self).__init__()
        self.p_list = p_list
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.leaky_relu1 = nn.LeakyReLU(negative_slope=0.1)

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.leaky_relu2 = nn.LeakyReLU(negative_slope=0.1)
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.leaky_relu3 = nn.LeakyReLU(negative_slope=0.1)
        
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm4 = nn.BatchNorm2d(128)
        self.leaky_relu4 = nn.LeakyReLU(negative_slope=0.1)

        self.conv5 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm5 = nn.BatchNorm2d(128)
        self.leaky_relu5 = nn.LeakyReLU(negative_slope=0.1)

        self.conv6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm6 = nn.BatchNorm2d(256)
        self.leaky_relu6 = nn.LeakyReLU(negative_slope=0.1)   

        self.conv7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm7 = nn.BatchNorm2d(256)
        self.leaky_relu7 = nn.LeakyReLU(negative_slope=0.1)   

        self.conv8 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm8 = nn.BatchNorm2d(256)
        self.leaky_relu8 = nn.LeakyReLU(negative_slope=0.1)

        self.conv9 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm9 = nn.BatchNorm2d(256)
        self.leaky_relu9 = nn.LeakyReLU(negative_slope=0.1)

        #self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.max_pool2d = nn.MaxPool2d(2, stride=2)
        self.linear_relu = nn.Sequential(
            nn.Linear(4096, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 100)
        )
        #self.softmax = nn.Softmax(dim=1)

        #self.relu = nn.ReLU()
        
    def quantize(self, X, p=0, NBIT=8):
        QRANGE = 2**(NBIT-1)
        p = torch.tensor(p,dtype=torch.int8).to('cpu')
        data_th = torch.clamp(X, -QRANGE*torch.float_power(2,-p), (QRANGE-1)*torch.float_power(2, -p))
        SCALE = torch.float_power(2,-p)
        data_qn = torch.round(data_th/SCALE)
        data_dqn = data_qn*SCALE
        return data_dqn
    
        
    def forward(self, x):
        out = self.batchnorm1(self.conv1(x))
        out = self.leaky_relu1(out)
        out = self.quantize(out, p=self.p_list[0])

        out = self.batchnorm2(self.conv2(out))
        out = self.leaky_relu2(out)
        out = self.quantize(out, p=self.p_list[1])
        out = self.max_pool2d(out)

        out = self.batchnorm3(self.conv3(out))
        out = self.leaky_relu3(out)        
        out = self.quantize(out, p=self.p_list[2])

        out = self.batchnorm4(self.conv4(out))
        out = self.leaky_relu4(out)
        out = self.quantize(out, p=self.p_list[3])

        out = self.batchnorm5(self.conv5(out))
        out = self.leaky_relu5(out)
        out = self.quantize(out, p=self.p_list[4])
        out = self.max_pool2d(out)

        out = self.batchnorm6(self.conv6(out))
        out = self.leaky_relu6(out)
        out = self.quantize(out, p=self.p_list[5])

        out = self.batchnorm7(self.conv7(out))
        out = self.leaky_relu7(out)
        out = self.quantize(out, p=self.p_list[6])

        out = self.batchnorm8(self.conv8(out))
        out = self.leaky_relu8(out)
        out = self.quantize(out, p=self.p_list[7])

        out = self.batchnorm9(self.conv9(out))
        out = self.leaky_relu9(out)
        out = self.quantize(out, p=self.p_list[8])
        out = self.max_pool2d(out)

        #out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.linear_relu(out)
        #out = self.softmax(out)
        
        return out

In [109]:
model_q = dongnet12q(cfg)
model_q.load_state_dict(torch.load('int8qmodel.pth'))
evaluate_model(model=model_q, test_loader=test_loader, device='cpu')

Dongnet12 cifar100 Accuracy: 0.6879
